In [83]:
import json
import azureml.core
from azureml.core import Workspace, Datastore, Dataset, Environment, Experiment
from azureml.data import FileDataset
from azureml.pipeline.core import Pipeline, PipelineData, PipelineParameter
from azureml.core.compute import ComputeTarget, AmlCompute


print(azureml.core.VERSION)
version = dict(zip(['major','minor','patch'], azureml.core.VERSION.split('.')))
ws = Workspace.from_config()

1.10.0


In [84]:
if int(version['major']) >= 1: 
    if int(version['minor']) == 10:
        from azureml.pipeline.steps import ParallelRunConfig, ParallelRunStep        
    else:
        from azureml.contrib.pipeline.steps import ParallelRunConfig, ParallelRunStep 

In [85]:
compute_name = "aml-compute1"
vm_size = "STANDARD_DS1_v2"
if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('Found compute target: ' + compute_name)
else:
    print('compute target not found, refer to 02_create_compute_cluster to create compute target...')

Found compute target: aml-compute1


In [86]:
dataset_name = 'anpr_images'
if dataset_name in ws.datasets: 
    anpr_images = ws.datasets.get(dataset_name)
    if anpr_images and type(anpr_images) is FileDataset:
        print('Found FileDataset: ' + dataset_name)
else:
    print('file dataset not found, refer to 01_config_notebook to setup file datasets...')


Found FileDataset: anpr_images


In [87]:
output_dir = PipelineData(name="scores", 
                          datastore=ws.get_default_datastore(), 
                          output_path_on_compute="batchscoring/results")

In [88]:
output_dir

$AZUREML_DATAREFERENCE_scores

# pipeline parameters

In [106]:
# kv = ws.get_default_keyvault()
# print(len(kv.get_secret(pipeline_kv_readapi)))
pipeline_inpart = PipelineParameter(name="pipeline_inpart", default_value='2020/07/28')
pipeline_kv_customimg = PipelineParameter(name="pipeline_kv_customimg", default_value='api-custom-vision')
pipeline_kv_readapi = PipelineParameter(name="pipeline_kv_readapi", default_value='api-readapi')



In [108]:
print(pipeline_inpart)

PipelineParameter_Name:pipeline_inpart_Default:2020/07/28


# python envinronment configuration 

In [90]:
# conda dependencies 
env_name = 'MAG-ParallelRunEnv'
print(env_name in ws.environments)
if env_name in ws.environments: 
    env = ws.environments.get(env_name)
    if env and type(env) is Environment: 
        print('Found environment: ' + env_name)
else: 
    print('environment not found, refer to 01_config_notebook to register envinroment...')


True
Found environment: MAG-ParallelRunEnv


# parallel run config and step

In [105]:
from azureml.data.datapath import DataPath, DataPathComputeBinding
print(pipeline_inpart.default_value)
datastore = Datastore(workspace=ws, name="godzilla")
print(datastore)
print(anpr_images.as_named_input('anpr_images').as_mount())
datapath = DataPath(datastore=datastore, path_on_datastore='images')
print(datapath.path_on_datastore)
data_path2 = (pipeline_inpart, DataPathComputeBinding(mode='mount'))
print(data_path2)


2020/07/28
{
  "name": "godzilla",
  "container_name": "batchml",
  "account_name": "godzillasinastorage",
  "protocol": "https",
  "endpoint": "core.windows.net"
}
images
(<azureml.pipeline.core.graph.PipelineParameter object at 0x0000016949E88820>, <azureml.data.datapath.DataPathComputeBinding object at 0x000001694B89EEB0>)


In [ ]:
# from azureml.core.datastore import Datastore
# from azureml.data.datapath import DataPath, DataPathComputeBinding
# from azureml.pipeline.steps import PythonScriptStep
# from azureml.pipeline.core import PipelineParameter

#    datastore = Datastore(workspace=workspace, name="godzilla")
#    datapath = DataPath(datastore=datastore, path_on_datastore='images')
#    data_path_pipeline_param = (PipelineParameter(name="input_data", default_value=datapath),
#                                DataPathComputeBinding(mode='mount'))

#    train_step = PythonScriptStep(script_name="train.py",
#                                  arguments=["--input", data_path_pipeline_param],
#                                  inputs=[data_path_pipeline_param],
#                                  target=compute_target,
#                                  source_directory=project_folder)

In [91]:
parallelrunconfig = ParallelRunConfig(
    environment=env, 
    entry_script='minibatch_process.py', 
    error_threshold=1,
    output_action='append_row', 
    compute_target=compute_target, 
    node_count=1, 
    process_count_per_node=2,
    mini_batch_size='2',
    source_directory='scripts', 
    description='description of batch step config',
    logging_level='INFO'
)


#parallelrunconfig('script', 'other stuff')
parallelrunstep = ParallelRunStep(
    name='cv-detection-batch-step', 
    parallel_run_config=parallelrunconfig, 
    inputs=[anpr_images.as_named_input('anpr_images').as_mount()], 
    # inputs=[ Dataset.File.from_files((godzilla_datastore, 'images')).as_named_input('anpr_images').as_mount()],
    # inputs=[]
    side_inputs=[], 
    output=output_dir,
    arguments=['--input_partition', pipeline_inpart, '--kv_customimage', pipeline_kv_customimg, '--kv_readapi', pipeline_kv_readapi], 
    allow_reuse=False
)


# ('config', 'inputs as mount', 'arguments passing in pipeline args')

In [92]:
parallelrunstep

# prepare pipeline 

In [93]:
#create pipeline
#publish pipeline 

pipeline = Pipeline(workspace=ws, steps=[parallelrunstep])
pipeline_run = Experiment(ws, 'MAG-batch-score1').submit(pipeline)
pipeline_run.wait_for_completion(show_output=True)

PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 92421688-cc32-4264-8b5b-baf6cd39b14a
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/MAG-batch-score1/runs/92421688-cc32-4264-8b5b-baf6cd39b14a?wsid=/subscriptions/907c8efc-c2c8-4c49-a4e1-aeb880e10c88/resourcegroups/aml/workspaces/magaml
StepRun( cv-detection-batch-step ) Status: NotStarted
StepRun( cv-detection-batch-step ) Status: Running

Streaming azureml-logs/55_azureml-execution-tvmps_3bc5f1fc037475a21e4bc34b5bc5effc525d3ae1a01005988b8cab813df8e7e6_d.txt
2020-07-29T13:00:58Z Executing 'Copy ACR Details file' on 10.0.0.4
2020-07-29T13:00:58Z Copy ACR Details file succeeded on 10.0.0.4. Output: 
>>>   
>>>   
2020-07-29T13:00:58Z Starting output-watcher...
2020-07-29T13:00:58Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_8c501e56007dc25550ab4b175a43b5f1
f7277927d38a: Pulling fs layer
8d3eac894

'Finished'